In [1]:
import re
import math
import numpy as np
import sys
import time

In [28]:


def textToState(text):
    inArr = text.split(';')
    numArr = []
    for num in inArr:
        num = re.sub("[^0-9]", "", num)
        numArr.append(num)
    size = int(math.sqrt(len(numArr)))
    stateArr = np.zeros([size,size], int)
    for i in range(len(numArr)):
        stateArr[math.floor(i/size)][math.floor(i%size)] = numArr[i]
    return tuple(tuple(row) for row in stateArr)
        
def readFile(fileName):
    f = open(fileName, 'r')
    i = 0
    
    sumLengthSol1 = 0
    sumLengthSol2 = 0
    sumLengthSol3 = 0
    sumLengthSol4 = 0
    sumLengthSearch1 = 0
    sumLengthSearch2 = 0
    sumLengthSearch3 = 0
    sumLengthSearch4 = 0
    sumNoSol1 = 0
    sumNoSol2 = 0
    sumNoSol3 = 0
    sumNoSol4 = 0
    sumTime1 = 0
    sumTime2 = 0
    sumTime3 = 0
    sumTime4 = 0
    stats = [[sumLengthSol1,sumLengthSol2,sumLengthSol3,sumLengthSol4],[sumLengthSearch1,sumLengthSearch2,sumLengthSearch3,sumLengthSearch4],[sumNoSol1,sumNoSol2,sumNoSol3,sumNoSol4],[sumTime1,sumTime2,sumTime3,sumTime4]]
    for line in f:
        stats = dfs(textToState(line), i, stats)
        stats = ids(textToState(line), 3, i, stats)
        stats = astar(textToState(line), 1, i, stats)
        stats = astar(textToState(line), 2, i, stats)
        print(stats)
        i = i+1
    f = open('stats.txt', "w")
    
    f.write('Depth First Search\n')
    f.write('Total Search Path: '+str(stats[1][0])+'\n')
    f.write('Average Search Path: '+str(stats[1][0]/i)+'\n')
    f.write('Total Solution Path: '+str(stats[0][0])+'\n')
    f.write('Average Solution Path: '+str(stats[0][0]/i)+'\n')
    f.write('No Solution: '+str(stats[2][0])+'\n')
    f.write('Total Solution Time: '+str(stats[3][0])+'\n')
    f.write('Average Solution Time: '+str(stats[3][0]/i)+'\n\n')
    
    f.write('Iterative Depth First Search\n')
    f.write('Total Search Path: '+str(stats[1][1])+'\n')
    f.write('Average Search Path: '+str(stats[1][1]/i)+'\n')
    f.write('Total Solution Path: '+str(stats[0][1])+'\n')
    f.write('Average Solution Path: '+str(stats[0][1]/i)+'\n')
    f.write('No Solution: '+str(stats[2][1])+'\n')
    f.write('Total Solution Time: '+str(stats[3][1])+'\n')
    f.write('Average Solution Time: '+str(stats[3][1]/i)+'\n\n')
    
    f.write('A* h1 Search\n')
    f.write('Total Search Path: '+str(stats[1][2])+'\n')
    f.write('Average Search Path: '+str(stats[1][2]/i)+'\n')
    f.write('Total Solution Path: '+str(stats[0][2])+'\n')
    f.write('Average Solution Path: '+str(stats[0][2]/i)+'\n')
    f.write('No Solution: '+str(stats[2][2])+'\n')
    f.write('Total Solution Time: '+str(stats[3][2])+'\n')
    f.write('Average Solution Time: '+str(stats[3][2]/i)+'\n\n')
    
    f.write('A* h2 Search\n')
    f.write('Total Search Path: '+str(stats[1][3])+'\n')
    f.write('Average Search Path: '+str(stats[1][3]/i)+'\n')
    f.write('Total Solution Path: '+str(stats[0][3])+'\n')
    f.write('Average Solution Path: '+str(stats[0][3]/i)+'\n')
    f.write('No Solution: '+str(stats[2][3])+'\n')
    f.write('Total Solution Time: '+str(stats[3][3])+'\n')
    f.write('Average Solution Time: '+str(stats[3][3]/i)+'\n\n')

class Node:
    def __init__(self, state, parent, depth, score):
        self.state = state
        self.parent = parent
        self.depth = depth
        self.score = 0
        self.children = []
    def setChildren(children):
        self.children = children
    def __str__(self):
        string = ''
        for i in range(len(self.state)):
            string = string + '{0}\n'.format(self.state[i])
        return string
    def explore(self, closedList, openList, goalState, heuristic):
        if self.state == goalState:
            closedList.append(self)
            return True, closedList, openList
        # Generate children
        size = len(self.state)
        childFrameArr = np.array(self.state)
        
        for i in range(size):
            for j in range(size):
                current = self.state[i][j]
                if i != size-1:
                    # create verical child
                    swap = self.state[i+1][j]
                    childArray = childFrameArr.copy()
                    childArray[i][j] = swap
                    childArray[i+1][j] = current
                    childState = tuple(tuple(row) for row in childArray)
                    child = Node(childState, self, self.depth + 1, 0)
                    self.children.append(child)
                    # Depth & Iterative
                    if(heuristic == None): 
                        exists = False
                        for openNode in openList:
                            if openNode.state == childState:
                                exists = True
                                break
                                #print('exists in open')
                        for closedNode in closedList:
                            if closedNode.state == childState or exists == True:
                                exists = True
                                break
                                #print('exists in closed')

                        if exists == False:
                            openList.insert(0,child)
                    # Heuristics
                    else:
                        pos = 0
                        score = 0
                        if heuristic == 1:
                            child.score = child.scoreh1() + child.depth
                            #print(child.score)
                        if heuristic == 2:
                            child.score = child.scoreh2() + child.depth
                        insert = True
                        for node in openList:
                            if child.state == node.state and child.depth > node.depth:
                                insert = False
                        for node in closedList:
                            if child.state == node.state:
                                insert = False
                        if insert == True:
                            openList.insert(i,child)
                if j != size-1:
                    # create horizontal child
                    swap = self.state[i][j+1]
                    childArray = childFrameArr.copy()
                    childArray[i][j] = swap
                    childArray[i][j+1] = current
                    childState = tuple(tuple(row) for row in childArray)
                    child = Node(childState, self, self.depth + 1, 0)
                    self.children.append(child)
                    # Depth & Iterative
                    if(heuristic == None): 
                        exists = False
                        for openNode in openList:
                            if openNode.state == childState:
                                exists = True
                                break
                                #print('exists in open')
                        for closedNode in closedList:
                            if closedNode.state == childState or exists == True:
                                exists = True
                                break
                                #print('exists in closed')

                        if exists == False:
                            openList.insert(0,child)
                    # Heuristics
                    else:
                        pos = 0
                        score = 0
                        if heuristic == 1:
                            child.score = child.scoreh1() + child.depth
                            #print(child.score)
                        if heuristic == 2:
                            child.score = child.scoreh2() + child.depth
                        
                        insert = True
                        for node in openList:
                            if child.state == node.state and child.depth > node.depth:
                                insert = False
                                break
                        for node in closedList:
                            if child.state == node.state:
                                insert = False
                                break
                        if insert == True:
                            openList.insert(i,child)
         
        openList.remove(self)
        closedList.append(self)
        #for child in range(len(self.children)):
            #print(self.children[child])
                    
        
        return False, closedList, openList
    
    def scoreh1(self):
        # in correct row? in correct column?
        score = 0
        size = len(self.state)
        for i in range(size):
            for j in range(size):
                number = self.state[i][j]
                # In wrong row?
                if number <= size * i or number > size * (i + 1):
                    score = score + 1
                # In wrong column?
                if (number-1) % size != j:
                    score = score + 1
                    
        return score
    def scoreh2(self):
        # Manhattan Distance
        score = 0
        size = len(self.state)
         
        for i in range(size):
            for j in range(size):
                number = self.state[i][j]
                correctRow = int(number / size)
                correctCol = (number-1) % size
                score = score + abs(i - correctRow) + abs(j - correctCol)
        return score

def getGoalState(size):
    goalStateArr = np.zeros([size,size], int)
    for i in range(size*size):
        goalStateArr[math.floor(i/size)][math.floor(i%size)] = i+1
    return tuple(tuple(row) for row in goalStateArr)
    
def dfs(initial, num, stats):
    start = time.time()
    check = 0
    initialNode = Node(initial, None, 0, 0)
    goalState = getGoalState(len(initialNode.state))
    openList = []
    openList.append(initialNode)
    closedList = []
    
    goal = False
    while openList != [] and goal == False and (check - start) < 60:
        goal, closedList, openList = openList[0].explore(closedList, openList, goalState, None)
        print(len(closedList))
        check = time.time()
    solutionNode = openList[0]
    solutionPath = []
    while solutionNode.parent != None:
        solutionPath.insert(-1,solutionNode)
        solutionNode = solutionNode.parent
    
    f = open("depth_first"+'_'+str(num)+'.txt', "w")
    f.write('Solution Path ('+str(len(solutionPath))+')\n\n')
    for node in solutionPath:
        f.write(node.__str__()+'\n')
    
    f.write('\nSearch Path ('+str(len(closedList)+1)+')\n\n')
    for node in closedList:
        f.write(node.__str__()+'\n')
    f.write(openList[0].__str__())
    
    if (check - start) > 60:
        stats[2][0] = stats[2][0] + 1
    else:
        stats[0][0] = stats[0][0] + len(solutionPath)
        stats[1][0] = stats[1][0] + len(closedList) + 1
        stats[3][0] = stats[3][0] + (check - start)
    return stats
    
def ids(initial, limit, num, stats):
    start = time.time()
    check = 0
    initialNode = Node(initial, None, 0, 0)
    goalState = getGoalState(len(initialNode.state))
    openList = []
    openList.append(initialNode)
    closedList = []
    overLimit = []
    limitTimes = 1
    
    goal = False
    while openList != [] and goal == False and (check - start) < 60: 
        if openList[0].depth >= limit * limitTimes:
            overLimit.append(openList[0])
            openList.remove(openList[0])
            #print('over limit')
            print(len(openList))
        else:
            goal, closedList, openList = openList[0].explore(closedList, openList, goalState,None)
            #print(len(closedList))
        if openList == [] and closedList != []:
            openList = overLimit.copy()
            overLimit = []
            limitTimes = limitTimes+1
            print('limit expanded')
        check = time.time()
        
    solutionNode = openList[0]
    solutionPath = []
    while solutionNode.parent != None:
        solutionPath.insert(-1,solutionNode)
        solutionNode = solutionNode.parent
    
    f = open("iterative_depth"+'_'+str(num)+'.txt', "w")
    f.write('Solution Path ('+str(len(solutionPath))+')\n\n')
    for node in solutionPath:
        f.write(node.__str__()+'\n')
    
    f.write('\nSearch Path ('+str(len(closedList)+1)+')\n\n')
    for node in closedList:
        f.write(node.__str__()+'\n')
    f.write(openList[0].__str__())
    
    if (check - start) > 60:
        stats[2][1] = stats[2][1] + 1
    else:
        stats[0][1] = stats[0][1] + len(solutionPath)
        stats[1][1] = stats[1][1] + len(closedList) + 1
        stats[3][1] = stats[3][1] + (check - start)
    return stats
    
    
def astar(initial, h, num, stats):
    start = time.time()
    check = 0
    initialNode = Node(initial, None, 0, 0)
    if h == 1:
        initialNode.score = initialNode.scoreh1()
    if h == 2:
        initialNode.score = initialNode.scoreh2()
    goalState = getGoalState(len(initialNode.state))
    openList = []
    openList.append(initialNode)
    closedList = []
    
    goal = False
    index = 0
    while openList != [] and goal == False and (check - start) < 60:
        mymin = sys.maxsize
        for i in range(len(openList)):
            if openList[i].score <= mymin:
                mymin = openList[i].score
                index = i
        print(openList[index].state)
        print(openList[index].score)
        if openList[index].state == goalState:
            goal = True
            break
        goal, closedList, openList = openList[index].explore(closedList, openList, goalState, h)
        check = time.time()
    
    solutionNode = openList[index]
    solutionPath = []
    while solutionNode.parent != None:
        solutionPath.insert(-1,solutionNode)
        solutionNode = solutionNode.parent
    
    f = open("astar"+str(h)+'_'+str(num)+'.txt', "w")
    f.write('Solution Path ('+str(len(solutionPath))+')\n\n')
    for node in solutionPath:
        f.write(node.__str__()+'\n')
    
    f.write('\nSearch Path ('+str(len(closedList)+1)+')\n\n')
    for node in closedList:
        f.write(node.__str__()+'\n')
    f.write(openList[index].__str__())
    
    if(h == 1):
        if (check - start) > 60:
            stats[2][2] = stats[2][2] + 1
        else:
            stats[0][2] = stats[0][2] + len(solutionPath)
            stats[1][2] = stats[1][2] + len(closedList) + 1
            stats[3][2] = stats[3][2] + (check - start)
    else:
        if (check - start) > 60:
            stats[2][3] = stats[2][3] + 1
        else:
            stats[0][3] = stats[0][3] + len(solutionPath)
            stats[1][3] = stats[1][3] + len(closedList) + 1
            stats[3][3] = stats[3][3] + (check - start)
    return stats
    
    


In [29]:
readFile('demoFile.txt')

1
2
3
4
31
30
29
28
27
26
25
24
23
22
21
30
29
28
27
26
25
24
23
22
21
20
28
27
26
25
24
23
22
21
20
19
28
27
26
25
24
23
22
21
20
19
18
26
25
24
23
22
21
20
19
18
17
25
24
23
22
21
20
19
18
17
16
24
23
22
21
20
19
18
17
16
15
23
22
21
20
19
18
17
16
15
14
22
21
20
19
18
17
16
15
14
13
21
20
19
18
17
16
15
14
13
12
20
19
18
17
16
15
14
13
12
11
30
29
28
27
26
25
24
23
22
21
20
28
27
26
25
24
23
22
21
20
19
27
26
25
24
23
22
21
20
19
18
27
26
25
24
23
22
21
20
19
18
17
25
24
23
22
21
20
19
18
17
16
25
24
23
22
21
20
19
18
17
16
15
23
22
21
20
19
18
17
16
15
14
22
21
20
19
18
17
16
15
14
13
21
20
19
18
17
16
15
14
13
12
20
19
18
17
16
15
14
13
12
11
19
18
17
16
15
14
13
12
11
10
28
27
26
25
24
23
22
21
20
19
18
27
26
25
24
23
22
21
20
19
18
17
25
24
23
22
21
20
19
18
17
16
24
23
22
21
20
19
18
17
16
15
23
22
21
20
19
18
17
16
15
14
23
22
21
20
19
18
17
16
15
14
13
21
20
19
18
17
16
15
14
13
12
20
19
18
17
16
15
14
13
12
11
19
18
17
16
15
14
13
12
11
10
18
17
16
15
14
13
12
11
10
9
26
25
